<a href="https://colab.research.google.com/github/woojin22/ds-section1-sprint2-new/blob/main/n123-Hypothesis-Test/n123a%20-%20Hypothesis%20Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://user-images.githubusercontent.com/6457691/89746244-a6e2a200-daf3-11ea-9314-7863d544d73c.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 1 / SPRINT 2 / NOTE 3*

---

# 📝 Assignment

# **🌱 기본 과제**

오늘 lecture note에서는 두 집단의 평균을 비교하는 가설 검정 과정을 부트스트래핑을 이용해 표본 평균 분포를 구현하여 통계적 유의미함을 살펴보았습니다. 

**T-test** 메소드에는 이러한 과정이 built-in 되어있기 때문에 우리는 동일한 계산을 **T-test** 메소드를 통해 간단히 구현해 볼 수 있습니다. 

## **Part 1. One-Sample T-test** 
한 샘플의 평균값과 특정값(모수)을 비교하는 검정 방법입니다. <br>
이를 양측검정을 통해 시행하고자 한다면 귀무가설과 대립가설은 다음과 같이 세울 수 있습니다. 

$H_0$ : $\mu_{sample} = \mu_{population}$

$H_a$ : $\mu_{sample} ≠ \mu_{population}$ 

### **1-1. 데이터셋 준비**
- 다음 데이터셋을 불러옵니다. <br>
  https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/seoul_tree/seoul_tree.txt
  - 데이터는 `tap`으로 구분되어 있습니다. 
  - `skiprows=1`로 설정하여 첫 번째 row가 헤더가 되도록 지정합니다. 
- `자치구`가 `서울시`, `공원녹지사업소`, `시설관리공단`인 데이터는 제외합니다. 
- `이팝나무` column만 추출하여 `df_sub`에 저장하세요. 
  - `df_sub`은 Series 형태입니다. 
- `df_sub`의 index를 재설정 하세요. 

##### **Q1. `df_sub`의 row의 총 개수를 쓰세요.**

In [2]:
import pandas as pd
import numpy as np

url = 'https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/seoul_tree/seoul_tree.txt'
df = pd.read_csv(url, sep = '\t', skiprows=1)
df = df[1:26]
df['이팝나무'] = df['이팝나무'].str.replace(',' , '').astype('int64')
df

,기간,자치구,노선수,수량,은행나무,양버즘나무,느티나무,왕벚나무,은단풍나무,메타세콰이어,...,튜울립나무,칠엽수,포플러,느릅나무,살구나무,무궁화,소나무,목련,이팝나무,기타
1,2019,종로구,70,"7,329","4,072","1,071",619,421,-,1,...,-,17,-,-,-,-,239,-,711,29
2,2019,중구,48,"8,250","3,640",162,460,289,-,-,...,7,-,-,-,21,46,"2,162",-,470,817
3,2019,용산구,39,"7,835","4,444","1,422",373,262,-,-,...,13,16,-,-,39,-,51,126,198,780
4,2019,성동구,38,"8,120","1,090","2,401","1,975",602,29,48,...,-,1,3,59,-,-,53,-,709,646
5,2019,광진구,28,"7,223","1,247","2,306","1,057",940,-,-,...,58,101,-,-,-,-,20,113,760,443
6,2019,동대문구,43,"10,664","3,749","3,342",855,747,-,60,...,37,19,-,1,26,1,-,-,1176,456
7,2019,중랑구,57,"8,365","2,774","2,759",471,354,-,74,...,-,-,-,-,-,212,64,-,449,"1,156"
8,2019,성북구,78,"8,449","2,722","1,282","1,014","1,656",66,65,...,4,-,-,-,9,5,97,-,853,547
9,2019,강북구,31,"7,789","3,510",934,324,497,284,-,...,5,-,-,140,-,-,"1,007",-,575,410
10,2019,도봉구,63,"9,572","4,914",323,779,"2,095",6,61,...,15,-,-,-,-,214,15,-,47,"1,070"


In [3]:
df_sub = df['이팝나무']
df_sub
df_sub.reset_index(drop=True)    # 정답은 25개

0      711
1      470
2      198
3      709
4      760
5     1176
6      449
7      853
8      575
9       47
10     142
11     443
12     493
13    1280
14     784
15    1426
16    1082
17     184
18    1281
19     397
20     249
21     446
22     768
23    1651
24     642
Name: 이팝나무, dtype: int64

### **1-2. 데이터 타입 변환**
- `이팝나무` column의 데이터 타입을 확인한 후, 적절한 데이터 타입으로 변환하세요.
  - 적절한 데이터 타입으로 변환하기 위해 `.replace()` 함수를 활용해보세요.

##### **Q2. `이팝나무` column의 데이터 타입으로 적절한 것은 무엇인가요?**

In [4]:
df_sub    # 정답은 int

1      711
2      470
3      198
4      709
5      760
6     1176
7      449
8      853
9      575
10      47
11     142
12     443
13     493
14    1280
15     784
16    1426
17    1082
18     184
19    1281
20     397
21     249
22     446
23     768
24    1651
25     642
Name: 이팝나무, dtype: int64

### **1-3. 표본 생성 및 가설 검정**
- [`np.random.choice()`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html)를 사용해 10개의 구의 `이팝나무` 데이터를 중복을 허용하여 추출합니다. 
    - `np.random.seed(123)`으로 설정하세요.
- 서울시 구별 `이팝나무` 데이터를 모집단으로 가정하였을 때, 10개의 구를 추출한 `이팝나무` 수의 평균과 모집단의 평균을 비교하는 가설검정을 One-Sample T-test를 통해 시행하세요.      
  - scipy 라이브러리의 [`.ttest_1samp()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_1samp.html)을 사용합니다.
- `.pvalue`를 사용하여 가설검정 결과의 P-value를 `pv1`에 저장하세요.

##### **Q3. 모집단(서울시 구별 이팝나무 데이터)의 평균은 얼마인지 소수점 둘째 자리까지 구하세요.**

In [5]:
import random
import scipy.stats as stats
from scipy.stats import ttest_1samp

np.random.seed(123)

구별이팝나무 = np.random.choice(df_sub, size = 10, replace=True)

t_stat, pv1 = stats.ttest_1samp(구별이팝나무, df_sub.mean())
#df_sub 는 이팝나무

표본평균 = 구별이팝나무.mean()
이팝나무평균 = df_sub.mean()
검정_통계량 = t_stat
P_value = pv1

이팝나무평균

688.64

##### **Q4. 표본(10개의 구를 추출한 이팝나무 데이터)의 평균은 얼마인지 소수점 첫째 자리까지 구하세요.**

In [6]:
구별이팝나무.mean()

479.7

##### **Q5. 가설 검정 결과 P-value를 해석한 것으로 알맞은 것은 무엇인가요?**
- 서울시 구별 이팝나무 수의 평균과 10개의 구를 샘플링한 이팝나무 수의 평균은 다르다고 볼 수 없다. 
- 서울시 구별 이팝나무 수의 평균과 10개의 구를 샘플링한 이팝나무 수의 평균은 같다고 볼 수 없다. 
- 서울시 구별 이팝나무 수의 평균은 10개의 구를 샘플링한 이팝나무 수의 평균보다 크다. 
- 서울시 구별 이팝나무 수의 평균은 10개의 구를 샘플링한 이팝나무 수의 평균보다 작다. 

In [23]:
round(P_value,3)
# 0.097 이므로 정답은 1번. 서울시 구별 이팝나무 수의 평균과 10개의 구를 샘플링한 이팝나무 수의 평균은 다르다고 볼 수 없다.

SyntaxError: ignored

# **🥇 도전 과제**

## **Part 2. Two-Sample T-test**


두 샘플의 평균값을 비교하는 검정 방법입니다. <br>
이를 양측검정을 통해 시행하고자 한다면 귀무가설과 대립가설은 다음과 같이 세울 수 있습니다. 

$H_0$ : $\mu_{sample1} = \mu_{sample2}$

$H_a$ : $\mu_{sample1} ≠ \mu_{sample2}$

In [134]:
# 아래의 코드를 실행해 주세요. 업그레이드 완료 후에는 런타임을 재시작해야 합니다. 
!pip install scipy --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 23.5 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3


In [1]:
# 버전 확인 
import scipy

scipy.__version__

'1.10.1'

### **2-1. 데이터셋 준비**
- **`서울시`, `공원녹지사업소`, `시설관리공단` 데이터는 제외하세요.** 
- **`느티나무`, `왕벚나무` column만 추출하여 각각 `zelkova`과 `cherry`에 저장하세요.** 

In [11]:
zelkova = df['느티나무']
cherry = df['왕벚나무']

1       619
2       460
3       373
4     1,975
5     1,057
6       855
7       471
8     1,014
9       324
10      779
11    1,165
12      680
13      721
14    3,056
15    1,065
16    2,231
17    1,884
18      407
19    3,419
20    1,421
21      185
22    3,409
23    2,991
24    2,400
25    2,417
Name: 느티나무, dtype: object

### **2-2. 데이터 타입 변환**
**`느티나무`, `왕벚나무` column의 데이터 타입을 `int`로 변환하세요.**

In [21]:
print(zelkova)
print(cherry)

1      619
2      460
3      373
4     1975
5     1057
6      855
7      471
8     1014
9      324
10     779
11    1165
12     680
13     721
14    3056
15    1065
16    2231
17    1884
18     407
19    3419
20    1421
21     185
22    3409
23    2991
24    2400
25    2417
Name: 느티나무, dtype: int64
1      421
2      289
3      262
4      602
5      940
6      747
7      354
8     1656
9      497
10    2095
11    1612
12    1748
13     732
14    1096
15     956
16    2508
17    2157
18    1123
19    2952
20    1127
21    2416
22     254
23    2455
24     992
25    2309
Name: 왕벚나무, dtype: int64


### **2-3. 가설 검정**
단측 검정(One-tailed test)을 사용하여 평균 `느티나무` 수와 평균 `왕벚나무` 수를 비교하는 가설검정을 시행하세요. (단, 서울시 구별 `느티나무` 및 `왕벚나무` 데이터는 표본입니다.) 
- **scipy 라이브러리의 [`.ttest_ind()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)를 사용합니다.**
  - One-tailed test를 진행하기 위해서는 `alternative` 파라미터를 설정해야 합니다. 
- **'평균 느티나무 수가 왕벚나무 수보다 통계적으로 유의미하게 크다'는 가설검정 결과의 P-value를 `pv2`에 저장하세요.**
   - 이에 알맞은 귀무가설과 대립가설을 설정합니다. 
- **'평균 왕벚나무 수가 느티나무 수보다 통계적으로 유의미하게 크다'는 가설검정 결과의 P-value를 `pv3`에 저장하세요.** 
  - 이에 알맞은 귀무가설과 대립가설을 설정합니다. 
- **평균 `느티나무` 수와 평균 `왕벚나무` 수를 비교하는 가설검정 결과를 `ans`에 저장하세요. 이때 `ans`는 다음 세 가지 중 하나입니다.** 
  - 느티나무 (평균 느티나무 수가 왕벚나무 수보다 통계적으로 유의하게 큰 경우)
  - 왕벚나무 (평균 왕벚나무 수가 느티나무 수보다 통계적으로 유의하게 큰 경우)
  - 같다. (통계적으로 유의미한 차이가 없다.) 

In [ ]:
# 이곳에 코드를 작성하세요.